In [1]:
import numpy as np

from human_terms_network import Human_Terms_Network
from dataset_load import *
from lr_baseline import LRBaseline

Using TensorFlow backend.


In [2]:
path = r"..\..\data\reviews_Amazon_Instant_Video_5.json.gz"

X, y = extract_review_amazon(path, 'reviewText')

y_label = y
# discard the review with 3. [1,2] = 0, [4,5] = 0
neutral_indices = np.where(y_label == 3)[0]
y_label[y_label<3] = 0
y_label[y_label>3] = 1

X_discarded = np.delete(X,neutral_indices)
y_discarded = np.delete(y_label, neutral_indices)

from sklearn.model_selection import train_test_split

X_train_split, X_test_split, y_train_amazon, y_test_amazon = train_test_split(
    X_discarded, y_discarded)

X_train_corpus_update = update_corpus_contraction(X_train_split)
X_test_corpus_update = update_corpus_contraction(X_test_split)



(75, 2)
corpus update start
corpus update end

(75, 2)
corpus update start
corpus update end



In [3]:
word_list, connotation = load_unigrams('./amazon-video-unigrams.txt', X_train_corpus_update, y_train_amazon)

In [4]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_corpus_update, X_test_corpus_update, 
                                                          word_list, connotation)

In [5]:
av_baseline = LRBaseline(X_train_corpus_update, X_test_corpus_update, y_train_amazon, y_test_amazon, 
                           human_terms=word_list)

In [6]:
# (train, test)

print(av_baseline.baseline())
print(av_baseline.baseline(penalty='l1'))
print(av_baseline.human_terms_baseline())
print(av_baseline.human_terms_baseline(penalty='l2'))

(0.9559585492227979, 0.9259259259259259)
(0.9554323186528497, 0.9284760170006071)
(0.9022425518134715, 0.9060109289617486)
(0.9022425518134715, 0.9060109289617486)


In [7]:
X_train = av_baseline.baseline_cv.transform(X_train_corpus_update)
X_test = av_baseline.baseline_cv.transform(X_test_corpus_update)
y_train = y_train_amazon
y_test = y_test_amazon

In [8]:
amazon_htm = Human_Terms_Network(input_shape=X_train.shape[1],
                               human_terms_shape=len(word_list),
                                domain='amazon')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1620)              0         
_________________________________________________________________
tanh_output (Dense)          (None, 1)                 1621      
Total params: 1,621
Trainable params: 1,621
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1620)         0                                            
_________________________________________

In [9]:
amazon_htm.set_data(X_train, X_test, y_train_agreement, y_test_agreement, y_train, y_test)

In [ ]:
amazon_htm.train(epochs=10)

Train on 16551 samples, validate on 8153 samples
Epoch 1/10
16551/16551 [==============================] - 44s 3ms/step - loss: 0.3577 - mean_absolute_error: 0.3716 - acc: 0.8076 - val_loss: 0.2741 - val_mean_absolute_error: 0.3502 - val_acc: 0.8008
Epoch 2/10
16551/16551 [==============================] - 43s 3ms/step - loss: 0.2568 - mean_absolute_error: 0.3220 - acc: 0.8221 - val_loss: 0.2515 - val_mean_absolute_error: 0.3248 - val_acc: 0.8177
Epoch 3/10
16551/16551 [==============================] - 43s 3ms/step - loss: 0.2322 - mean_absolute_error: 0.2994 - acc: 0.8379 - val_loss: 0.2406 - val_mean_absolute_error: 0.3072 - val_acc: 0.8320
Epoch 4/10
16551/16551 [==============================] - 43s 3ms/step - loss: 0.2186 - mean_absolute_error: 0.2853 - acc: 0.8461 - val_loss: 0.2341 - val_mean_absolute_error: 0.2963 - val_acc: 0.8401
Epoch 5/10
16551/16551 [==============================] - 43s 3ms/step - loss: 0.2098 - mean_absolute_error: 0.2760 - acc: 0.8518 - val_loss: 0.229

In [ ]:
amazon_htm.combined.evaluate([X_test, amazon_htm.y_test_agreement], amazon_htm.y_test)

In [ ]:
amazon_htm.test(reject=True)

In [ ]:
amazon_htm.X_test.shape

In [ ]:
amazon_htm.y_train_agreement.shape